# Prompt, Mask, Train, Save, Pipeline-Predict

#### prompt template: 2

# Notebook:

##### Imports
##### replace 'MajorClaim' by 'Stance'
##### make train and test datasets
##### train BERT Mask on the train dataset
##### save the model
##### create a pipeline using the saved model
##### run predictions on it
##### evaluate with the test set

### Imports

In [1]:
!pip install transformers
!pip install ipywidgets
!pip install IProgress
!pip install pandas
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import transformers
import torch

In [3]:
from transformers import BertTokenizer, BertForMaskedLM

import numpy as np

import datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import load_metric

In [4]:
import pandas as pd

### tokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### make the train, test dataframe

In [6]:
df = pd.read_pickle("/notebooks/Prompting/notebooks/link_task_prompting/pe_dataset_w_prompts_2_linkID_pickle")

In [7]:
# rename MajorClaim as stance

In [8]:
df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5968,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...
5969,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...
5970,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...,Which of these choices best describes the fol

In [9]:
df.columns

Index(['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_x',
       'label_ComponentType', 'relation_SupportAttack', 'label_RelationType',
       'label_LinkedNotLinked', 'split', 'essay', 'argument_bound_1',
       'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr',
       'total_paras', 'token_count', 'token_count_covering_para',
       'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count',
       'succeeding_tokens_in_sentence_count', 'token_ratio',
       'relative_position_in_para_char', 'is_in_intro',
       'relative_position_in_para_token', 'is_in_conclusion',
       'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para',
       'nr_following_comps_in_para', 'structural_fts_as_text',
       'structural_fts_as_text_combined', 'para_ratio', 'first_or_last',
       'strct_fts_w_position_in_essay', 'component_pos_tags',
       'strct_fts_essay_position_pos_tags', 'prompted_representation_2'],
      dtype='object')

In [10]:
df['prompted_representation_2'][0]

'Which of these choices best describes the following component? "Linked" or "Separate". Component: Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., First or last in essay: Yes, First in paragraph: Yes, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: PRON, VERB, VERB, ADJ, NOUN, ADP, NOUN, ADP, ADJ, NOUN. Linked.'

In [11]:
def rename_not_linked(x):
    
    comp = x.prompted_representation_2
    
    new_comp = comp.replace("NotLinked.", "Separate.")
    
    return new_comp

In [12]:
rename_not_linked(df.iloc[3])

'Which of these choices best describes the following component? "Linked" or "Separate". Component: Topic: Should students be taught to compete or to cooperate?, Sentence: What we acquired from team work is not only how to achieve the same goal with others but more importantly, how to get along with others, First or last in essay: No, First in paragraph: No, Last in paragraph: No, In in introduction: No, Is in conclusion: No. Part Of Speech tags: PRON, PRON, VERB, ADP, NOUN, NOUN, AUX, PART, ADV, ADV, PART, VERB, DET, ADJ, NOUN, ADP, NOUN, CCONJ, ADV, ADV, PUNCT, ADV, PART, AUX, ADP, ADP, NOUN. Separate.'

In [13]:
df['prompted_representation_2_corrected'] = df.apply(lambda x: rename_not_linked(x), axis=1)

In [14]:
df['prompted_representation_2_corrected'][3]

'Which of these choices best describes the following component? "Linked" or "Separate". Component: Topic: Should students be taught to compete or to cooperate?, Sentence: What we acquired from team work is not only how to achieve the same goal with others but more importantly, how to get along with others, First or last in essay: No, First in paragraph: No, Last in paragraph: No, In in introduction: No, Is in conclusion: No. Part Of Speech tags: PRON, PRON, VERB, ADP, NOUN, NOUN, AUX, PART, ADV, ADV, PART, VERB, DET, ADJ, NOUN, ADP, NOUN, CCONJ, ADV, ADV, PUNCT, ADV, PART, AUX, ADP, ADP, NOUN. Separate.'

In [15]:
# now do a sanity check to see if it's correct

In [16]:
last_tokens_list = []

In [17]:
def get_last_token(x):
    
    prompted_rep = x.prompted_representation_2_corrected
    
    last_token = prompted_rep.split()[-1]
    
    last_tokens_list.append(last_token)

In [18]:
df.apply(lambda x: get_last_token(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
5968    None
5969    None
5970    None
5971    None
5972    None
Length: 5973, dtype: object

In [19]:
last_tokens_list

['Linked.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Linked.',
 'Linked.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Separate.',
 'Linked.',
 'Separate.',
 'Separate.',


In [20]:
set(last_tokens_list)

{'Linked.', 'Separate.'}

In [21]:
####### YAAAAAAAAASSSSSSSSSS KINGGGGGGGGGGGGGGGG # correct.

## make train, test datasets

### train dataset

In [22]:
df_train = df[df.split == 'TRAIN']

In [23]:
df_train = df_train.reset_index(drop=True)

In [24]:
df_train

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_2_corrected
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1,essay001,T2,MajorClaim 2154 2231,a more cooperative attitudes towards life is m...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,1.00,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADV, ADJ, NOUN, ADP,...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
2,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
3,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
4,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4708,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
4709,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
4710,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,

In [25]:
df_train.split.value_counts()

TRAIN    4713
Name: split, dtype: int64

In [26]:
prompted_texts_train_full = df_train['prompted_representation_2_corrected'][:].tolist()

In [27]:
len(prompted_texts_train_full)

4713

In [28]:
4713 * .2

942.6

In [29]:
import random

In [30]:
prompted_texts_val = random.sample(prompted_texts_train_full, 472)

In [31]:
len(prompted_texts_val)

472

In [32]:
prompted_texts_train = [x for x in prompted_texts_train_full if x not in prompted_texts_val]

In [33]:
len(prompted_texts_train)

4235

In [34]:
len(prompted_texts_train) + len(prompted_texts_val)

4707

In [35]:
### prepare train dataset

In [36]:
inputs_train = tokenizer(prompted_texts_train, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [37]:
inputs_train

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [38]:
inputs_train['input_ids'].shape

torch.Size([4235, 512])

In [39]:
# for training we will need labels

In [40]:
inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [41]:
inputs_train

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
      

In [42]:
inputs_train['labels'].shape

torch.Size([4235, 512])

In [43]:
# find where the last 102 is

In [44]:
(inputs_train['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

122

In [45]:
tokenizer.decode(4366)

'c l a i m'

In [46]:
tokenizer.decode(18458)

'p r e m i s e'

In [47]:
# claim = 4366, premise = 18458

In [48]:
(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

146

In [49]:
inputs_train['input_ids'][1][151]

tensor(0)

In [50]:
# confirm that all at [len-2] indices are either claim or premise

In [51]:
sep_indices_list = []

for i in range(len(prompted_texts_train)):
    sep_idx = (inputs_train['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list.append(sep_idx)

In [52]:
sep_indices_list

[122,
 146,
 172,
 221,
 151,
 102,
 153,
 289,
 115,
 141,
 155,
 144,
 114,
 189,
 140,
 150,
 125,
 150,
 120,
 130,
 177,
 136,
 163,
 144,
 106,
 141,
 180,
 152,
 132,
 120,
 209,
 142,
 129,
 132,
 222,
 162,
 154,
 137,
 120,
 125,
 128,
 133,
 145,
 137,
 199,
 181,
 171,
 149,
 171,
 144,
 171,
 166,
 128,
 105,
 122,
 108,
 111,
 93,
 112,
 134,
 110,
 132,
 101,
 125,
 99,
 129,
 132,
 157,
 103,
 127,
 141,
 139,
 129,
 178,
 113,
 140,
 152,
 153,
 118,
 134,
 144,
 116,
 127,
 106,
 114,
 117,
 149,
 164,
 118,
 125,
 100,
 162,
 117,
 131,
 161,
 119,
 118,
 103,
 116,
 110,
 127,
 97,
 131,
 232,
 216,
 103,
 121,
 147,
 111,
 128,
 146,
 145,
 151,
 120,
 124,
 117,
 120,
 103,
 108,
 143,
 102,
 96,
 187,
 134,
 155,
 131,
 133,
 135,
 125,
 129,
 151,
 154,
 124,
 140,
 123,
 128,
 150,
 106,
 150,
 117,
 120,
 156,
 105,
 106,
 148,
 144,
 113,
 166,
 147,
 152,
 151,
 117,
 197,
 138,
 105,
 146,
 146,
 107,
 178,
 146,
 127,
 129,
 175,
 128,
 139,
 190,
 110,
 1

In [53]:
len(sep_indices_list)

4235

In [54]:
class_tokens_indices_list = [x - 2 for x in sep_indices_list]

In [55]:
class_tokens_indices_list

[120,
 144,
 170,
 219,
 149,
 100,
 151,
 287,
 113,
 139,
 153,
 142,
 112,
 187,
 138,
 148,
 123,
 148,
 118,
 128,
 175,
 134,
 161,
 142,
 104,
 139,
 178,
 150,
 130,
 118,
 207,
 140,
 127,
 130,
 220,
 160,
 152,
 135,
 118,
 123,
 126,
 131,
 143,
 135,
 197,
 179,
 169,
 147,
 169,
 142,
 169,
 164,
 126,
 103,
 120,
 106,
 109,
 91,
 110,
 132,
 108,
 130,
 99,
 123,
 97,
 127,
 130,
 155,
 101,
 125,
 139,
 137,
 127,
 176,
 111,
 138,
 150,
 151,
 116,
 132,
 142,
 114,
 125,
 104,
 112,
 115,
 147,
 162,
 116,
 123,
 98,
 160,
 115,
 129,
 159,
 117,
 116,
 101,
 114,
 108,
 125,
 95,
 129,
 230,
 214,
 101,
 119,
 145,
 109,
 126,
 144,
 143,
 149,
 118,
 122,
 115,
 118,
 101,
 106,
 141,
 100,
 94,
 185,
 132,
 153,
 129,
 131,
 133,
 123,
 127,
 149,
 152,
 122,
 138,
 121,
 126,
 148,
 104,
 148,
 115,
 118,
 154,
 103,
 104,
 146,
 142,
 111,
 164,
 145,
 150,
 149,
 115,
 195,
 136,
 103,
 144,
 144,
 105,
 176,
 144,
 125,
 127,
 173,
 126,
 137,
 188,
 108,
 118

In [56]:
# inputs_pe['input_ids'][0][143].item() # correct

In [57]:
# now get a list of input ids at all these indics: they should be either claim = 4366, premise = 18458

In [58]:
list_index_minus_2 = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    list_index_minus_2.append(at_idx_minus_2)

In [59]:
set(list_index_minus_2)

{3584, 5799}

In [60]:
tokenizer.decode(3584)

's e p a r a t e'

In [61]:
tokenizer.decode(5799)

'l i n k e d'

In [62]:
# correct

In [63]:
# now make the class token indices 103

In [64]:
for idx, val in enumerate(class_tokens_indices_list):
    inputs_train['input_ids'][idx][val] = 103

In [65]:
# now check if they are 103

In [66]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_train['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [67]:
set(unsontroi_check_list)

{103}

In [68]:
# correct

In [69]:
class PeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [70]:
train_dataset = PeDataset(inputs_train)

In [71]:
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=)
# maybe we don't need this.

### prepare val dataset

In [72]:
inputs_eval = tokenizer(prompted_texts_val, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [73]:
inputs_eval

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [74]:
inputs_eval['input_ids'].shape

torch.Size([472, 512])

In [75]:
# for training we will need labels

In [76]:
inputs_eval['labels'] = inputs_eval.input_ids.detach().clone()

In [77]:
inputs_eval

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
      

In [78]:
inputs_eval['labels'].shape

torch.Size([472, 512])

In [79]:
# find where the last 102 is

In [80]:
(inputs_eval['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

121

In [81]:
tokenizer.decode(4366)

'c l a i m'

In [82]:
tokenizer.decode(18458)

'p r e m i s e'

In [83]:
# claim = 4366, premise = 18458

In [84]:
(inputs_eval['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

130

In [85]:
inputs_eval['input_ids'][1][123]

tensor(4748)

In [86]:
# confirm that all at [len-2] indices are either claim or premise

In [87]:
sep_indices_list = []

for i in range(len(prompted_texts_val)):
    sep_idx = (inputs_eval['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list.append(sep_idx)

In [88]:
sep_indices_list

[121,
 130,
 155,
 136,
 144,
 167,
 133,
 154,
 229,
 177,
 107,
 115,
 113,
 126,
 111,
 111,
 121,
 131,
 134,
 121,
 98,
 144,
 118,
 189,
 120,
 162,
 162,
 154,
 101,
 158,
 143,
 151,
 105,
 122,
 126,
 131,
 141,
 140,
 140,
 103,
 110,
 133,
 187,
 113,
 133,
 141,
 103,
 126,
 160,
 124,
 113,
 139,
 128,
 122,
 115,
 157,
 149,
 105,
 151,
 107,
 166,
 120,
 106,
 196,
 154,
 122,
 121,
 167,
 131,
 168,
 140,
 120,
 129,
 137,
 115,
 131,
 128,
 100,
 215,
 140,
 191,
 126,
 133,
 141,
 147,
 109,
 127,
 153,
 160,
 113,
 105,
 124,
 116,
 156,
 134,
 116,
 137,
 156,
 138,
 128,
 112,
 209,
 162,
 136,
 132,
 215,
 115,
 143,
 130,
 111,
 154,
 111,
 132,
 124,
 125,
 158,
 129,
 121,
 134,
 162,
 187,
 102,
 127,
 119,
 221,
 158,
 177,
 96,
 119,
 131,
 188,
 145,
 142,
 101,
 139,
 117,
 147,
 136,
 131,
 97,
 177,
 194,
 128,
 132,
 122,
 181,
 137,
 122,
 200,
 127,
 115,
 99,
 200,
 121,
 114,
 264,
 150,
 130,
 177,
 125,
 124,
 89,
 116,
 180,
 111,
 156,
 141,
 11

In [89]:
len(sep_indices_list)

472

In [90]:
class_tokens_indices_list = [x - 2 for x in sep_indices_list]

In [91]:
class_tokens_indices_list

[119,
 128,
 153,
 134,
 142,
 165,
 131,
 152,
 227,
 175,
 105,
 113,
 111,
 124,
 109,
 109,
 119,
 129,
 132,
 119,
 96,
 142,
 116,
 187,
 118,
 160,
 160,
 152,
 99,
 156,
 141,
 149,
 103,
 120,
 124,
 129,
 139,
 138,
 138,
 101,
 108,
 131,
 185,
 111,
 131,
 139,
 101,
 124,
 158,
 122,
 111,
 137,
 126,
 120,
 113,
 155,
 147,
 103,
 149,
 105,
 164,
 118,
 104,
 194,
 152,
 120,
 119,
 165,
 129,
 166,
 138,
 118,
 127,
 135,
 113,
 129,
 126,
 98,
 213,
 138,
 189,
 124,
 131,
 139,
 145,
 107,
 125,
 151,
 158,
 111,
 103,
 122,
 114,
 154,
 132,
 114,
 135,
 154,
 136,
 126,
 110,
 207,
 160,
 134,
 130,
 213,
 113,
 141,
 128,
 109,
 152,
 109,
 130,
 122,
 123,
 156,
 127,
 119,
 132,
 160,
 185,
 100,
 125,
 117,
 219,
 156,
 175,
 94,
 117,
 129,
 186,
 143,
 140,
 99,
 137,
 115,
 145,
 134,
 129,
 95,
 175,
 192,
 126,
 130,
 120,
 179,
 135,
 120,
 198,
 125,
 113,
 97,
 198,
 119,
 112,
 262,
 148,
 128,
 175,
 123,
 122,
 87,
 114,
 178,
 109,
 154,
 139,
 110,


In [92]:
# inputs_pe['input_ids'][0][143].item() # correct

In [93]:
# now get a list of input ids at all these indics: they should be either claim = 4366, premise = 18458

In [94]:
list_index_minus_2 = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_eval['input_ids'][idx][val].item()
    list_index_minus_2.append(at_idx_minus_2)

In [95]:
set(list_index_minus_2)

{3584, 5799}

In [96]:
# correct

In [97]:
# now make the class token indices 103

In [98]:
for idx, val in enumerate(class_tokens_indices_list):
    inputs_eval['input_ids'][idx][val] = 103

In [99]:
# now check if they are 103

In [100]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_list):
    at_idx_minus_2 = inputs_eval['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [101]:
set(unsontroi_check_list)

{103}

In [102]:
# correct

In [103]:
# class PeDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings
#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#     def __len__(self):
#         return len(self.encodings.input_ids)

In [104]:
eval_dataset = PeDataset(inputs_eval)

### test dataset

In [105]:
df_test = df[df.split == 'TEST']

In [106]:
df_test = df_test.reset_index(drop=True)

In [107]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_2_corrected
0,essay004,T1,MajorClaim 262 376,this industry has affected the cultural attrib...,MajorClaim,MajorClaim,[],,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, DET...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1,essay004,T2,MajorClaim 1663 1758,the tourism has created threatening pressure o...,MajorClaim,MajorClaim,[],,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,1.00,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, VER...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
2,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,1,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
3,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
4,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.75,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,2,Topic: We can not forcedly put the same number...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1256,essay398,T13,Premise 1595 1648,this could avoid imbalance of gender in some s...,Premise,Premise,[],Support,NotLinked,TEST,...,1,Topic: We can not forcedly put the same number...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: DET, VERB, VERB, NOUN, AD...",Topic: We can not forcedly put the same number...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1257,essay398,T14,Premise 1650 1734,It would affect students' mental health to stu...,Premise,Premise,[],Support,NotLinked,

In [108]:
df_test.split.value_counts()

TEST    1260
Name: split, dtype: int64

In [109]:
prompted_texts_test = df_test['prompted_representation_2_corrected'][:].tolist()

In [110]:
len(prompted_texts_test)

1260

In [111]:
inputs_test = tokenizer(prompted_texts_test, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [112]:
inputs_test

{'input_ids': tensor([[ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0],
        [ 101, 2029, 1997,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [113]:
inputs_test['input_ids'].shape

torch.Size([1260, 512])

In [114]:
# for test we do not need labels

In [115]:
#inputs_train['labels'] = inputs_train.input_ids.detach().clone()

In [116]:
#inputs_train

In [117]:
#inputs_train['labels'].shape

In [118]:
# find where the last 102 is

In [119]:
#(inputs_train['input_ids'][0] == 102).nonzero(as_tuple=True)[0].item()

In [120]:
#tokenizer.decode(4366)

In [121]:
#tokenizer.decode(18458)

In [122]:
# claim = 4366, premise = 18458

In [123]:
#(inputs_train['input_ids'][1] == 102).nonzero(as_tuple=True)[0].item()

In [124]:
#inputs_train['input_ids'][1][169]

In [125]:
# confirm that all at [len-2] indices are either claim or premise

In [126]:
sep_indices_list_test = []

for i in range(len(prompted_texts_test)):
    sep_idx = (inputs_test['input_ids'][i] == 102).nonzero(as_tuple=True)[0].item()
    sep_indices_list_test.append(sep_idx)

In [127]:
sep_indices_list_test

[147,
 129,
 130,
 113,
 100,
 139,
 255,
 146,
 124,
 232,
 151,
 116,
 142,
 155,
 138,
 141,
 201,
 148,
 132,
 107,
 114,
 115,
 118,
 95,
 142,
 129,
 135,
 139,
 106,
 135,
 107,
 120,
 123,
 109,
 145,
 136,
 104,
 139,
 102,
 128,
 128,
 112,
 115,
 123,
 116,
 95,
 103,
 119,
 114,
 89,
 99,
 162,
 124,
 145,
 105,
 137,
 152,
 142,
 104,
 116,
 143,
 146,
 149,
 128,
 159,
 139,
 192,
 175,
 221,
 166,
 166,
 140,
 120,
 140,
 113,
 129,
 144,
 111,
 133,
 139,
 123,
 148,
 156,
 146,
 132,
 104,
 110,
 142,
 156,
 103,
 100,
 107,
 120,
 130,
 120,
 157,
 133,
 147,
 117,
 125,
 107,
 183,
 96,
 172,
 143,
 139,
 141,
 129,
 117,
 227,
 136,
 148,
 144,
 109,
 131,
 183,
 148,
 133,
 112,
 124,
 131,
 161,
 189,
 130,
 182,
 98,
 200,
 98,
 153,
 112,
 158,
 126,
 136,
 164,
 114,
 157,
 111,
 130,
 95,
 128,
 164,
 113,
 132,
 108,
 138,
 161,
 197,
 115,
 114,
 120,
 143,
 181,
 130,
 129,
 126,
 133,
 152,
 108,
 148,
 159,
 208,
 139,
 116,
 126,
 146,
 140,
 116,
 146,


In [128]:
len(sep_indices_list_test)

1260

In [129]:
class_tokens_indices_test_list = [x - 2 for x in sep_indices_list_test]

In [130]:
class_tokens_indices_test_list

[145,
 127,
 128,
 111,
 98,
 137,
 253,
 144,
 122,
 230,
 149,
 114,
 140,
 153,
 136,
 139,
 199,
 146,
 130,
 105,
 112,
 113,
 116,
 93,
 140,
 127,
 133,
 137,
 104,
 133,
 105,
 118,
 121,
 107,
 143,
 134,
 102,
 137,
 100,
 126,
 126,
 110,
 113,
 121,
 114,
 93,
 101,
 117,
 112,
 87,
 97,
 160,
 122,
 143,
 103,
 135,
 150,
 140,
 102,
 114,
 141,
 144,
 147,
 126,
 157,
 137,
 190,
 173,
 219,
 164,
 164,
 138,
 118,
 138,
 111,
 127,
 142,
 109,
 131,
 137,
 121,
 146,
 154,
 144,
 130,
 102,
 108,
 140,
 154,
 101,
 98,
 105,
 118,
 128,
 118,
 155,
 131,
 145,
 115,
 123,
 105,
 181,
 94,
 170,
 141,
 137,
 139,
 127,
 115,
 225,
 134,
 146,
 142,
 107,
 129,
 181,
 146,
 131,
 110,
 122,
 129,
 159,
 187,
 128,
 180,
 96,
 198,
 96,
 151,
 110,
 156,
 124,
 134,
 162,
 112,
 155,
 109,
 128,
 93,
 126,
 162,
 111,
 130,
 106,
 136,
 159,
 195,
 113,
 112,
 118,
 141,
 179,
 128,
 127,
 124,
 131,
 150,
 106,
 146,
 157,
 206,
 137,
 114,
 124,
 144,
 138,
 114,
 144,
 1

In [131]:
# inputs_pe['input_ids'][0][143].item() # correct

In [132]:
# now get a list of input ids at all these indices: they should be either claim = 4366, premise = 18458

In [133]:
list_index_minus_2_test = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    list_index_minus_2_test.append(at_idx_minus_2)

In [134]:
set(list_index_minus_2)

{3584, 5799}

In [135]:
# correct

In [136]:
# now make the class token indices 103

In [137]:
for idx, val in enumerate(class_tokens_indices_test_list):
    inputs_test['input_ids'][idx][val] = 103

In [138]:
# now check if they are 103

In [139]:
unsontroi_check_list = []

for idx, val in enumerate(class_tokens_indices_test_list):
    at_idx_minus_2 = inputs_test['input_ids'][idx][val].item()
    unsontroi_check_list.append(at_idx_minus_2)

In [140]:
set(unsontroi_check_list)

{103}

In [141]:
# correct

In [142]:
# class PeDataset(torch.utils.data.Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings
#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#     def __len__(self):
#         return len(self.encodings.input_ids)

In [143]:
test_dataset = PeDataset(inputs_test)

In [144]:
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True)
# maybe we don't need this.

## get the model

In [145]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [146]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [147]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [148]:
# let's see with a simple trainer now and then we will try the fancy trainer.

### training

In [149]:
from transformers import TrainingArguments

In [150]:
import gc

In [151]:
gc.collect()
torch.cuda.empty_cache()

In [152]:
NB_EPOCHS = 6
BATCH_SIZE = 8

In [153]:
training_args = TrainingArguments(
    
    # output
    output_dir='output',          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=1e-5,#2e-5,                 # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
    # eval
    evaluation_strategy="epoch",              # cf. paper Sun et al.
    #eval_steps=20,                            # cf. paper Sun et al.
    #eval_accumulation_steps = BATCH_SIZE,
    # fp16_full_eval = True,
    
    # clean the tensor board folder to see the loss properly.
    
    # log
    logging_dir="/notebooks/Prompting/results",  
    logging_strategy='steps',
    logging_steps=100,
    
    # save
    save_strategy='epoch',
    save_total_limit=2,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    metric_for_best_model='eval_loss' 
    # metric_for_best_model='f1'
)

In [154]:
# from transformers import TrainingArguments

# args = TrainingArguments(
#     output_dir = 'out',
#     per_device_train_batch_size=16,
#     num_train_epochs=2
# )

In [155]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
    #compute_metrics=compute_metrics
)

In [156]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4235
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3180
/tmp/ipykernel_165/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,0.004600,0.002490
2,0.002000,0.001340
3,0.001500,0.001161
4,0.001400,0.001065
5,0.001300,0.001104
6,0.001200,0.001048


***** Running Evaluation *****
  Num examples = 472
  Batch size = 8
Saving model checkpoint to output/checkpoint-530
Configuration saved in output/checkpoint-530/config.json
Model weights saved in output/checkpoint-530/pytorch_model.bin
/tmp/ipykernel_165/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 472
  Batch size = 8
Saving model checkpoint to output/checkpoint-1060
Configuration saved in output/checkpoint-1060/config.json
Model weights saved in output/checkpoint-1060/pytorch_model.bin
/tmp/ipykernel_165/1625588428.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.t

TrainOutput(global_step=3180, training_loss=0.3468991641986108, metrics={'train_runtime': 1752.3838, 'train_samples_per_second': 14.5, 'train_steps_per_second': 1.815, 'total_flos': 6688034293248000.0, 'train_loss': 0.3468991641986108, 'epoch': 6.0})

In [157]:
trainer.save_model("pe_mask_model_prompted_rep_2_linkedID")

Saving model checkpoint to pe_mask_model_prompted_rep_2_linkedID
Configuration saved in pe_mask_model_prompted_rep_2_linkedID/config.json
Model weights saved in pe_mask_model_prompted_rep_2_linkedID/pytorch_model.bin


In [156]:
# new idea: take this model and finetune the PE dataset on it.
# so instead of loading a pre-trained BERT, load this instead.

### Pipeline

In [158]:
saved_model = BertForMaskedLM.from_pretrained("/notebooks/Prompting/notebooks/link_task_prompting/pe_mask_model_prompted_rep_2_linkedID")

loading configuration file /notebooks/Prompting/notebooks/link_task_prompting/pe_mask_model_prompted_rep_2_linkedID/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.21.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /notebooks/Prompting/notebooks/link_task_prompting/pe_mask_model_prompted_rep_2_linkedID/pytorch_model.bin
All model checkpoint weights were used when initializin

In [159]:
from transformers import pipeline

In [160]:
classifier = pipeline("fill-mask", model=saved_model, tokenizer=tokenizer)

In [161]:
classifier.model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [162]:
targets = ['separate', 'linked']

In [163]:
# ok now see how to do this for the whole list of test components

In [164]:
# but first we must replace the last token by [MASK]!

In [165]:
def get_masked_components(x):
    
    prompted_rep = x.prompted_representation_2_corrected
    
    last_token = prompted_rep.split()[-1]
    

    new_rep = prompted_rep.replace(last_token, "[MASK].")

    
    return new_rep

In [166]:
df_test

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split,...,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags,prompted_representation_2,prompted_representation_2_corrected
0,essay004,T1,MajorClaim 262 376,this industry has affected the cultural attrib...,MajorClaim,MajorClaim,[],,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, DET...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1,essay004,T2,MajorClaim 1663 1758,the tourism has created threatening pressure o...,MajorClaim,MajorClaim,[],,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,1.00,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, AUX, VERB, VER...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
2,essay004,T3,Claim 179 239,the tourism bring large profit for the destina...,Claim,Claim,[],Attack,NotLinked,TEST,...,1,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.25,1,Topic: International tourism is now more commo...,"Part Of Speech tags: DET, NOUN, VERB, ADJ, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
3,essay004,T4,Claim 953 1031,international tourism can create negative impa...,Claim,Claim,[],Support,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.50,0,Topic: International tourism is now more commo...,"Part Of Speech tags: ADJ, NOUN, VERB, VERB, AD...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
4,essay004,T5,Claim 1578 1624,tourism has threatened the nature environments,Claim,Claim,[],Support,Linked,TEST,...,0,Topic: International tourism is now more commo...,Topic: International tourism is now more commo...,0.75,0,Topic: International tourism is now more commo...,"Part Of Speech tags: NOUN, AUX, VERB, DET, NOU...",Topic: International tourism is now more commo...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,essay398,T12,Claim 1484 1589,universities should encourage more girls to ch...,Claim,Claim,[],Support,Linked,TEST,...,2,Topic: We can not forcedly put the same number...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: NOUN, VERB, VERB, ADJ, NO...",Topic: We can not forcedly put the same number...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1256,essay398,T13,Premise 1595 1648,this could avoid imbalance of gender in some s...,Premise,Premise,[],Support,NotLinked,TEST,...,1,Topic: We can not forcedly put the same number...,Topic: We can not forcedly put the same number...,0.80,0,Topic: We can not forcedly put the same number...,"Part Of Speech tags: DET, VERB, VERB, NOUN, AD...",Topic: We can not forcedly put the same number...,Which of these choices best describes the foll...,Which of these choices best describes the foll...
1257,essay398,T14,Premise 1650 1734,It would affect students' mental health to stu...,Premise,Premise,[],Support,NotLinked,

In [167]:
df_test['masked_representation_2_for_pipeline'] = df_test.apply(lambda x: get_masked_components(x), axis=1)

In [168]:
df_test['masked_representation_2_for_pipeline'][0]

'Which of these choices best describes the following component? "Linked" or "Separate". Component: Topic: International tourism is now more common than ever before, Sentence: While some might think the tourism bring large profit for the destination countries, I would contend that this industry has affected the cultural attributes and damaged the natural environment of the tourist destinations., First or last in essay: Yes, First in paragraph: No, Last in paragraph: Yes, In in introduction: Yes, Is in conclusion: No. Part Of Speech tags: DET, NOUN, AUX, VERB, DET, ADJ, NOUN, CCONJ, VERB, DET, ADJ, NOUN, ADP, DET, NOUN, NOUN. [MASK].'

In [169]:
# ok now do a sanity check to see that they are done correctly

In [170]:
last_tokens_mask_list = []

In [171]:
def get_last_token(x):
    
    prompted_rep = x.masked_representation_2_for_pipeline
    
    last_token = prompted_rep.split()[-1]
    
    last_tokens_mask_list.append(last_token)

In [172]:
df_test.apply(lambda x: get_last_token(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1255    None
1256    None
1257    None
1258    None
1259    None
Length: 1260, dtype: object

In [173]:
last_tokens_mask_list

['[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[MASK].',
 '[M

In [174]:
set(last_tokens_mask_list)

{'[MASK].'}

In [175]:
###### YASSSSS SIRRRRRRRRRRR # correct

In [176]:
# ok now see how we can give all these 1260 statements to the pipeline

In [177]:
test_components_list = df_test['masked_representation_2_for_pipeline'][:].tolist()

In [178]:
len(test_components_list)

1260

In [179]:
# ok now give it to the pipeline

In [180]:
classifier_output = classifier(test_components_list, targets=targets)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


In [181]:
classifier_output[0][0]['token']

3584

In [182]:
predicted_labels_test = []

for idx in range(1260):
    
    predicted_token = classifier_output[idx][0]['token']
    predicted_labels_test.append(predicted_token)

In [183]:
len(predicted_labels_test)

1260

In [184]:
# sanity check
set(predicted_labels_test)

{3584, 5799}

In [185]:
# ok correct

In [186]:
test_labels_str = df_test.label_LinkedNotLinked[:].tolist()

In [187]:
test_labels_str

['Linked',
 'Linked',
 'NotLinked',
 'Linked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'Linked',
 'Linked',
 'Linked',
 'Linked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'NotLinked',
 'Linked',
 'Linked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'NotLinked',
 'Linked',
 'Linked',
 'NotLinked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'Linked',
 'Linked',
 'Linked',
 'Linked',
 'NotLinked',
 'NotLinked',
 'NotLinked',
 'NotLinked',

In [188]:
set(test_labels_str)

{'Linked', 'NotLinked'}

In [189]:
test_labels_int = []

In [190]:
def get_int_labels():
    
    for idx, val in enumerate(test_labels_str):
        
        if val == 'Linked':
            
            test_labels_int.append(5799)
            
        elif val == 'NotLinked':
            
            test_labels_int.append(3584)

In [191]:
get_int_labels()

In [192]:
test_labels_int

[5799,
 5799,
 3584,
 5799,
 5799,
 3584,
 3584,
 3584,
 3584,
 3584,
 5799,
 5799,
 5799,
 5799,
 5799,
 5799,
 3584,
 5799,
 3584,
 3584,
 3584,
 3584,
 3584,
 5799,
 5799,
 3584,
 5799,
 5799,
 5799,
 3584,
 3584,
 5799,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 5799,
 3584,
 3584,
 3584,
 5799,
 5799,
 5799,
 3584,
 3584,
 5799,
 5799,
 5799,
 3584,
 3584,
 3584,
 3584,
 5799,
 3584,
 3584,
 5799,
 3584,
 3584,
 5799,
 5799,
 3584,
 5799,
 5799,
 3584,
 5799,
 3584,
 3584,
 3584,
 3584,
 5799,
 5799,
 5799,
 5799,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 5799,
 3584,
 5799,
 3584,
 5799,
 5799,
 5799,
 5799,
 3584,
 3584,
 3584,
 5799,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 3584,
 5799,
 5799,
 3584,
 5799,
 5799,
 3584,
 3584,
 3584,
 5799,
 3584,
 3584,
 5799,
 5799,
 5799,
 5799,
 5799,
 3584,
 3584,
 3584,
 3584,
 5799,
 3584,
 5799,
 5799,
 3584,
 3584,
 3584,
 3584,
 5799,
 3584,
 3584,
 3584,
 5799,
 5799,
 5799,
 5799,
 5799,

In [193]:
len(test_labels_int)

1260

In [194]:
set(test_labels_int)

{3584, 5799}

In [195]:
# now do the classification report

In [196]:
from sklearn.metrics import classification_report

In [197]:
print(classification_report(test_labels_int, predicted_labels_test, digits=3))

              precision    recall  f1-score   support

        3584      0.795     0.840     0.817       744
        5799      0.749     0.688     0.717       516

    accuracy                          0.778      1260
   macro avg      0.772     0.764     0.767      1260
weighted avg      0.776     0.778     0.776      1260

